In [3]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nagch\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nagch\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [5]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [6]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [7]:
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [6]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
10/10 [==============================] - 0s 698us/step - loss: 2.3061 - accuracy: 0.0851
Epoch 2/200
10/10 [==============================] - 0s 900us/step - loss: 2.2001 - accuracy: 0.1277
Epoch 3/200
10/10 [==============================] - 0s 598us/step - loss: 2.0710 - accuracy: 0.1915
Epoch 4/200
10/10 [==============================] - 0s 798us/step - loss: 1.9990 - accuracy: 0.3830
Epoch 5/200
10/10 [==============================] - 0s 798us/step - loss: 1.9581 - accuracy: 0.3830
Epoch 6/200
10/10 [==============================] - 0s 698us/step - loss: 1.7611 - accuracy: 0.6170
Epoch 7/200
10/10 [==============================] - 0s 997us/step - loss: 1.6220 - accuracy: 0.5532
Epoch 8/200
10/10 [==============================] - 0s 798us/step - loss: 1.5806 - accuracy: 0.5532
Epoch 9/200
10/10 [==============================] - 0s 698us/step - loss: 1.4170 - accuracy: 0.5532
Epoch 10/200
10/10 [==============================] - 0s 997us/step - loss: 1.1981 - accura

10/10 [==============================] - 0s 698us/step - loss: 0.0400 - accuracy: 1.0000
Epoch 82/200
10/10 [==============================] - 0s 598us/step - loss: 0.1141 - accuracy: 0.9787
Epoch 83/200
10/10 [==============================] - 0s 698us/step - loss: 0.0481 - accuracy: 1.0000
Epoch 84/200
10/10 [==============================] - 0s 499us/step - loss: 0.0282 - accuracy: 1.0000
Epoch 85/200
10/10 [==============================] - 0s 798us/step - loss: 0.0152 - accuracy: 1.0000
Epoch 86/200
10/10 [==============================] - 0s 598us/step - loss: 0.0209 - accuracy: 1.0000
Epoch 87/200
10/10 [==============================] - 0s 698us/step - loss: 0.0513 - accuracy: 0.9787
Epoch 88/200
10/10 [==============================] - 0s 698us/step - loss: 0.0312 - accuracy: 1.0000
Epoch 89/200
10/10 [==============================] - 0s 499us/step - loss: 0.0551 - accuracy: 0.9787
Epoch 90/200
10/10 [==============================] - 0s 698us/step - loss: 0.0657 - accuracy: 

10/10 [==============================] - 0s 798us/step - loss: 0.0065 - accuracy: 1.0000
Epoch 161/200
10/10 [==============================] - 0s 598us/step - loss: 0.0042 - accuracy: 1.0000
Epoch 162/200
10/10 [==============================] - 0s 798us/step - loss: 0.0710 - accuracy: 0.9787
Epoch 163/200
10/10 [==============================] - 0s 499us/step - loss: 0.0117 - accuracy: 1.0000
Epoch 164/200
10/10 [==============================] - 0s 798us/step - loss: 0.0056 - accuracy: 1.0000
Epoch 165/200
10/10 [==============================] - 0s 499us/step - loss: 0.0061 - accuracy: 1.0000
Epoch 166/200
10/10 [==============================] - 0s 698us/step - loss: 0.0606 - accuracy: 0.9787
Epoch 167/200
10/10 [==============================] - 0s 499us/step - loss: 0.0178 - accuracy: 1.0000
Epoch 168/200
10/10 [==============================] - 0s 698us/step - loss: 0.0140 - accuracy: 1.0000
Epoch 169/200
10/10 [==============================] - 0s 598us/step - loss: 0.0560 - a

In [8]:
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [9]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [11]:
msg1 = ''
while msg1 != 'END':
    msg1 = input("")
    response = chatbot_response(msg1)
    print(response)

Hi
Hi there, how can I help?
Need some info
Hello, thanks for asking
How can you help me?
Offering support for Adverse drug reaction, Blood pressure, Hospitals and Pharmacies
What are the nearest Pharmacies
Please provide pharmacy name
CVS
Hello, thanks for asking
END
Hi there, how can I help?
